In [1]:
import sys 
import os 
import re 
# import requests
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np 
import os
import json
import subprocess
from datetime import datetime

# from pathlib import Path
# script_location = Path(__file__).absolute().parent
# target_directory = script_location / 'mentalLLM' / 'config.py'

sys.path.append("/Users/ngokykhanhthu/Documents/Mental_Health_n_LLMs")
from mentalLLM.config import RAW_DIR, INTERIM_DIR, PROCESSED_DIR, FINAL_DIR, DATASET, DATA_KEYWORD, CHAT_GPT_QUESTION

from chat_gpt_scraper import remove_white_spaces, is_unix_time, convert_unix_time, find_links, scrape_link, retrieve_the_json_portion, extract_json_object, create_csv_files, extract_msg

In [2]:
def check_if_chat_message(unix_stamp, data): 
    """
    helper method for create_csv_files
    given unix stamp and the json list, figure out if it is gibberish or chat message
   
    Args:
        unix_stamp (float): an unix timestamp 
        data (list): the json file 

    Returns: 
        True if it is a valid chat message
        False otherwise  
    """
    # TODO 
    return True 

In [3]:
def extract_msg(unix_stamp, data): 
    """
    helper method for create_csv_files
    given unix stamp and the json list, extract chat message
    Args:
        unix_stamp (float): an unix timestamp 
        data (list): the json file 

    Returns: 
        message
    """
    curr_index = data.index(unix_stamp) + 1
    strings = []
    
    entered_string = False 
    while curr_index < len(data):
        curr_element = data[curr_index]
        if isinstance(curr_element, str):
            strings.append(curr_element)  
            curr_index += 1
        elif not strings: 
            curr_index += 1 
        else: 
            curr_index = len(data)

    return repr(str(strings[-1]))
      

In [4]:
def classify_sender(message): 
    """
    helper method for create_csv_files
    given unix stamp and the json list, figure out who sent message 
     Args:
        message (str): a message extracted 

    Returns: 
        str: chatgpt/user 
    """
    msg_raw = message.strip('\'')
    print("messsage before stripping: ", message)

    print("message after stripping: ", msg_raw)

    if re.match(r"^'", msg_raw):
        return "chatgpt" 

    return "user"
    # return bool(re.match(r'.*("\""|"\'").*', message[0]))
    # message_raw = repr(message)[1:]
    # state =  bool(re.fullmatch(r'.*[\']".*', message_raw))
    # if state: 
    #     return "chatgpt"
    # return "user" 


In [5]:
def create_csv_files(file_path): 
    """
    given json file path n order of file,
    extracts important information and outputs csv file
    
    Args:
        file_path (os.path): path to json file 
        order_of_file (int): how to number the resulting file 
  
    """

    # load json file 
    with open(file_path, "r", encoding='utf-8', errors='strict') as f:
        data = json.load(f)

    # initialise df 
    rows = []
    
    # meta data stuff 
    title = ""
    create_time = 0 
    update_time = 0

    # go through the json list 
    for i in range(0,len(data)-1, 1): 
        sender = ""
        # extract title 
        if isinstance(data[i], str) and "title" in data[i]: 
            title = data[i+1]
            continue

        if not is_unix_time(data[i]): 
            continue 

        timestamp = data[i]

        # extract file create time 
        if create_time == 0 : 
            create_time = convert_unix_time(timestamp)
            continue 
            
        # extract file latest update time 
        elif update_time == 0:     
            update_time = convert_unix_time(timestamp)
            continue 
    
        # extract timestamp + message 
        
        # if there is a timestamp right afterwards, probs a chatgpt response 
        elif i + 1 < len(data) and is_unix_time(data[i+1]): 
            # print("data at ", i, "is ", data[i], "anddd second cond state is ", is_unix_time(data[i+1]))

            timestamp =  data[i+1]
            sender = "chatgpt"
            
        # deal w the extra timestamp
        elif  i - 1  > 0 and is_unix_time(data[i-1]) : 
            continue

        # if there is not, check if gibberish. 
        elif not check_if_chat_message(timestamp, data): 
            continue

        # if not, get the message 
        message = extract_msg(timestamp, data) 

        if 'sender' != "": 
            sender = classify_sender(message)

        # update rows 
        rows.append([title, 
                        create_time, 
                        update_time,
                        sender, 
                        convert_unix_time(timestamp), 
                        message])
    
    df = pd.DataFrame(rows, columns=['title', 'create_time', "update_time", 'sender', 'timestamp', 'message'])
    df.to_csv(f"../extracted_chat.csv", quotechar = "'", index=False, encoding='utf-8')
    # df.to_csv(f"{FINAL_DIR}/{order_of_file}_extracted_chat.csv", index=False)


In [6]:
create_csv_files(f"../{PROCESSED_DIR}/8_json_output.json")

messsage before stripping:  'how to build an LLM model from scratch? '
message after stripping:  how to build an LLM model from scratch? 
messsage before stripping:  'parent_id'
message after stripping:  parent_id
messsage before stripping:  'model_set_context'
message after stripping:  model_set_context
messsage before stripping:  'thoughts'
message after stripping:  thoughts
messsage before stripping:  '{search_query: [{q:how to train a large language model from scratch guide 2024 2025}, {q:state of the art LLM training libraries DeepSpeed Megatron-DeepSpeed FlashAttention 2024}, {q:compute and data requirements to train a 7B 70B parameter model 2024 blog}, {q:tokenizers Byte-Pair Encoding SentencePiece WordPiece tokenization for LLMs 2024}, {q:Hugging Face transformers training from scratch tutorial 2024}], response_length: short}'
message after stripping:  {search_query: [{q:how to train a large language model from scratch guide 2024 2025}, {q:state of the art LLM training librarie